#Load required Python libraries, LIBRA modules, and set Matplotlib to display figures inside the notebook.

In [ ]:
import os
import glob
import numpy as np
import scipy.sparse as sp
from libra_py import units, data_stat, influence_spectrum
import matplotlib.pyplot as plt
from liblibra_core import *
from libra_py.workflows.nbra import step3
import libra_py.packages.cp2k.methods as CP2K_methods
%matplotlib notebook

#Excitation energies in SP and MB basis

Read excitation energies from Hvib files, convert to eV, reference all energies to state 0, and plot energy versus time for SP and MB basis.

In [ ]:

params = {"path_to_energy_files": "res-mb-sd-DFT", "dt": 1.0, 
          "prefix": "Hvib_sd_", "suffix": "_re", "istep": 1000, "fstep": 4999}

titles = ['SP basis', 'MB basis']
plt.figure()
for c, basis in enumerate(['sd','ci']):
    plt.subplot(1,2,c+1)
    params.update({"prefix": F"Hvib_{basis}_"})
    md_time, energies = CP2K_methods.extract_energies_sparse(params)
    energies = energies * units.au2ev
    print(energies.shape)
    for i in range(energies.shape[1]):
        plt.plot(md_time, energies[:,i]-energies[:,0])
    
    plt.title(titles[c])
    plt.ylabel('Excitation energy, eV')
    plt.xlabel('Time, fs')
    plt.ylim(-0.3, 4.3)
plt.tight_layout()

#NACs heatmaps 

Text: Load NAC matrices for all time steps, take absolute values, average over time, convert to meV, and plot the averaged NAC matrix for SP and MB basis.

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import glob
import scipy.sparse as sp

titles = ['SP', 'MB']
plt.figure()

global_max = 0
nstates_max = 0

for basis in ['sd', 'ci']:
    nac_files = glob.glob(f'res-mb-sd-DFT/Hvib_{basis}*im*')
    for nac_file in nac_files:
        nac_mat = sp.load_npz(nac_file).todense().real
        global_max = max(global_max, np.abs(nac_mat).max())
        nstates_max = max(nstates_max, nac_mat.shape[0])

vmin, vmax = 0, global_max * 1000  # Scale to meV

for c1, basis in enumerate(['sd', 'ci']):
    plt.subplot(1, 2, c1 + 1)
    
    nac_ave = None
    nac_files = glob.glob(f'res-mb-sd-DFT/Hvib_{basis}*im*')
    
    for c2, nac_file in enumerate(nac_files):
        nac_mat = sp.load_npz(nac_file).todense().real
        if c2 == 0:
            nac_ave = np.zeros_like(nac_mat)
        nac_ave += np.abs(nac_mat)
    
    nac_ave *= 1000 * units.au2ev / len(nac_files) 
    nstates = nac_ave.shape[0]
    
    # Plot the averaged NAC matrix
    plt.imshow(
        np.flipud(nac_ave),
        cmap='hot',
        extent=(0, nstates_max, 0, nstates_max),  
        vmin=vmin,
        vmax=vmax
    )
    plt.xlabel('State index')
    plt.ylabel('State index')
    plt.colorbar(shrink=0.50).ax.set_title('meV')
    plt.title(f'{titles[c1]} NACs')

plt.tight_layout()
plt.show()


#PDOS

Text: Compute the average PDOS from all PDOS files, align energies to the HOMO, and plot element and orbital resolved PDOS together with the total DOS.

In [ ]:

%matplotlib notebook
params = {"path_to_all_pdos": 'path/to/all_pdosfiles', "atoms": [[1,2,3,4] , ['Cu', 'Zn' ,'Sn' ,'S']],
          "orbitals_cols": [[3], range(4,7), range(7,12), range(12,19)], "orbitals":  ['s','p','d','f'],
          "npoints": 4000, "sigma": 0.05, "shift": 2.0}

ave_energy_grid, homo_energy, ave_pdos_convolved, pdos_labels, ave_pdos_convolved_total = CP2K_methods.pdos(params)
print(pdos_labels)
for i in range(len(pdos_labels)):
    pdos_label = pdos_labels[i]
    plt.plot(ave_energy_grid-homo_energy, ave_pdos_convolved[i], label=pdos_label)
plt.plot(ave_energy_grid-homo_energy, ave_pdos_convolved_total, color='black', label='Total')
plt.legend()
plt.xlim(-6,6)
plt.ylabel('pDOS, 1/eV')
plt.xlabel('Energy, eV')
plt.title('average Pdos for dft+u, 300 K')
plt.tight_layout()

#CI coefficients with error bars

Text: Analyze the dominant configurations for each excited state and plot the CI weights with error bars to show the statistical spread.

In [ ]:
# %matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt

params = {
    "path_to_logfiles": "/path/to/all_logfiles",  # adjust
    "number_of_states": 130,
    "tolerance": 0.01,
    "isUKS": 0,
    "nsds": 4
}


colors = ['green', 'blue', 'red', 'purple']

coeffs, errors = CP2K_methods.exc_analysis(params)


plt.rcParams.update({
    'font.family': 'Times New Roman',
    'font.size': 14,
    'font.weight': 'bold',
    'axes.labelweight': 'bold',
    'axes.titlesize': 14,
    'axes.titleweight': 'bold',
    'axes.linewidth': 1.5,
    'xtick.major.width': 1.5,
    'ytick.major.width': 1.5,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'legend.fontsize': 12  
})

fig, ax = plt.subplots(figsize=(5.5, 4.5))  # match your other figures

n_states = len(coeffs)
for state in range(n_states):
    for conf in range(params["nsds"]):
        ax.plot(
            state + 1, coeffs[state][conf],
            color="black",
            marker='s',
            markerfacecolor=colors[conf],
            markeredgewidth=0.6,
            markersize=6
        )
        ax.errorbar(
            state + 1, coeffs[state][conf],
            yerr=errors[state][conf],
            color='black',
            capsize=2,
            linewidth=1
        )

# Labels
ax.set_xlabel("Electronic State Index", fontweight='bold')
ax.set_ylabel(r"$c_i^2$ (Probability Weight)", fontweight='bold')
ax.grid(False)

# Legend handles & labels
legend_handles = [
    plt.Line2D([0], [0], color='black', marker='s',
               markerfacecolor=colors[i], linestyle='None', markersize=7)
    for i in range(params["nsds"])
]
legend_labels = [f"Configuration {i+1}" for i in range(params["nsds"])]

ax.legend(
    legend_handles, legend_labels,
    loc='upper right', bbox_to_anchor=(0.98, 0.98),
    ncol=2,           
    frameon=False,
    borderaxespad=0.0,
    prop={'size': 10},     
    handlelength=1.2, handletextpad=0.4,
    columnspacing=0.8, labelspacing=0.3
)

plt.tight_layout()
plt.savefig("ci_coefficients_czts.jpg", dpi=600, bbox_inches='tight')
plt.show()


#TRPES and energy relaxation fitting

Text: Compute TRPES from dynamics outputs, track the peak or mean energy versus time, and fit the relaxation using an exponential model to extract a characteristic time.

In [ ]:
import warnings
from libra_py import trpes
%matplotlib inline
warnings.filterwarnings("ignore") 

# Data locations (EDIT THESE)

METHODS = {
    "FSSH":  "/path/to/FSSH_NBRA_icond_",
    "FSSH2": "/path/to/FSSH2_NBRA_icond_",
    "IDA":   "/path/to/IDA_NBRA_icond_",
}

EPREFIX = "/path/to/dft_logfiles/step_"
ESUFFIX = ".log"

def process_trpes(method_name: str, dprefix: str):
    # Use "x" as the parameter dict (matches your original notation)
    x = {
        "istep": 1000,
        "fstep": 4998,
        "dt": 1.0,
        "namd_nsteps": 3998,
        "units": "eV",
        "eprefix": EPREFIX,
        "esuffix": ESUFFIX,
        "input_file_type": 1,
        "logfile_read_params": {"number_of_states": 130},
        "iconds": [0, 4000, 400],
        "dprefix": dprefix,
        "dsuffix": "/mem_data.hdf",
        "de": 0.02,
        "emin": 0.0,
        "emax": 5.0,
        "sigma_e": 0.1,
    }


    time_bins, energy_grid, H_total = trpes.compute_trpes(x)
    time_ps = time_bins / 1000.0

    peak_energies = np.array([
        energy_grid[np.argmax(H_total[i, :])]
        for i in range(H_total.shape[0])
    ])

    # Double exponential decay
    def double_exp(t, A1, tau1, A2, tau2, C):
        return A1 * np.exp(-t / tau1) + A2 * np.exp(-t / tau2) + C

    p0 = [
        (peak_energies[0] - peak_energies[-1]) * 0.7,  # A1
        0.5,                                           # tau1
        (peak_energies[0] - peak_energies[-1]) * 0.3,  # A2
        3.0,                                           # tau2
        peak_energies[-1],                             # C
    ]

    try:
        popt, _ = curve_fit(double_exp, time_ps, peak_energies, p0=p0, maxfev=5000)
        fit_curve = double_exp(time_ps, *popt)

        # Define relaxation times as tau*ln(3)
        t_relax1 = popt[1] * np.log(3)
        t_relax2 = popt[3] * np.log(3)

        # Weighted relaxation time by amplitudes
        total_amp = abs(popt[0]) + abs(popt[2])
        t_relax = (abs(popt[0]) * t_relax1 + abs(popt[2]) * t_relax2) / total_amp

    except RuntimeError:
        # Fallback: single exponential
        def single_exp(t, A, tau, C):
            return A * np.exp(-t / tau) + C

        popt, _ = curve_fit(
            single_exp, time_ps, peak_energies,
            p0=[peak_energies[0] - peak_energies[-1], 1.0, peak_energies[-1]]
        )
        fit_curve = single_exp(time_ps, *popt)
        t_relax = popt[1] * np.log(3)

    return time_ps, energy_grid, H_total, fit_curve, t_relax, method_name


def make_figure(methods: dict, outfile_prefix: str = "trpes_3panel_vertical"):
    fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(7, 10), constrained_layout=True)

    panel_letters = ["(a)", "(b)", "(c)"]
    items = list(methods.items())

    im = None
    for i, ax in enumerate(axs):
        method, path = items[i]
        time_ps, energy_grid, H_total, fit_curve, t_relax, _ = process_trpes(method, path)

        im = ax.imshow(
            H_total.T,
            origin="lower",
            aspect="auto",
            extent=[time_ps[0], time_ps[-1], energy_grid[0], energy_grid[-1]],
        )

        ax.plot(time_ps, fit_curve, color="white", linewidth=1.0)

        ax.text(
            0.02, 0.94, f"t_relax = {t_relax:.2f} ps",
            transform=ax.transAxes,
            color="white",
            path_effects=[path_effects.withStroke(linewidth=2, foreground="black")]
        )

        ax.text(-0.12, 1.05, panel_letters[i], transform=ax.transAxes, clip_on=False)

        ax.set_ylabel("Energy (eV)")
        if i == len(axs) - 1:
            ax.set_xlabel("Time (ps)")

    # Shared colorbar
    cbar = fig.colorbar(im, ax=axs, location="right")
    cbar.set_label("Intensity")

    fig.savefig(f"{outfile_prefix}.pdf", bbox_inches="tight")
    fig.savefig(f"{outfile_prefix}.png", bbox_inches="tight")
    plt.show()


if __name__ == "__main__":
    make_figure(METHODS)
